In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math
from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor

In [83]:
import numpy as np
import pandas as pd
import os
%matplotlib inline

INPUT_DIR = "dataset"

#os.listdir(INPUT_DIR)

#os.chdir(INPUT_DIR)
expense = pd.read_csv("promotional_expense.csv")
train = pd.read_csv("yds_train2018.csv")
test = pd.read_csv("yds_test2018.csv")
holiday = pd.read_excel("holidays.xlsx")

In [84]:
import re
holiday['Date'] = holiday.Date.apply(lambda x: re.sub(r'\s','',x))

holiday.Date = pd.to_datetime(holiday.Date, format="%Y,%m,%d")

holiday["Year"] = holiday.Date.apply(lambda x: x.year)
holiday["Month"] = holiday.Date.apply(lambda x: x.month)
holiday["Day"] = holiday.Date.apply(lambda x: x.day)

In [85]:
holiday_grouped = holiday.groupby(by = ['Year','Month','Country'], as_index=False)

In [86]:
holiday_grouped = holiday_grouped.agg({"Holiday":"count"})

In [87]:
expense = expense.rename(columns = {"Product_Type":"Product_ID"})
train = pd.merge(train, expense, how='left', on= ['Year','Month','Country','Product_ID'])

In [88]:
train_grouped = train.groupby(by=['Year','Month','Country','Product_ID'], as_index=False)
train_grouped = train_grouped.agg({"Sales":"sum", "Expense_Price":"mean"})

test = pd.merge(test, expense, how="left", on=['Year','Month','Country','Product_ID'])
test_grouped = test.groupby(by=['Year','Month','Country','Product_ID'], as_index=False)
test_grouped = test_grouped.agg({"Sales":"sum","Expense_Price":"mean"})

In [89]:
# train_grouped['Expense_Price'] = train_grouped['Expense_Price'].fillna(train_grouped['Expense_Price'].mean())
# test_grouped['Expense_Price'] = test_grouped['Expense_Price'].fillna(test_grouped['Expense_Price'].mean())

In [90]:
train_grouped = pd.merge(train_grouped, holiday_grouped, how='left', on=['Year','Month','Country'])
test_grouped = pd.merge(test_grouped, holiday_grouped, how='left', on=['Year','Month','Country'])

In [91]:
train_3 = train_grouped[train_grouped['Product_ID'] == 3]
train_not3 = train_grouped[train_grouped['Product_ID'] != 3]
test_3 = test_grouped[test_grouped['Product_ID'] == 3]
test_not3 = test_grouped[test_grouped['Product_ID'] != 3]

In [92]:
train_not3['Holiday'] = train_not3['Holiday'].fillna(0)
test_not3['Holiday'] = test_not3['Holiday'].fillna(0)
train_3['Holiday'] = train_3['Holiday'].fillna(0)
test_3['Holiday'] = test_3['Holiday'].fillna(0)

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [93]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
label_en = LabelEncoder()
label_en.fit(list(train_grouped['Country'].values)+list(test_grouped['Country'].values))
train_grouped['Country'] = label_en.transform(list(train_grouped['Country'].values))
test_grouped['Country'] = label_en.transform(list(test_grouped['Country'].values))

In [94]:
train_grouped['Holiday'] = train_grouped['Holiday'].fillna(0)
test_grouped['Holiday'] = test_grouped['Holiday'].fillna(0)

In [140]:
import lightgbm as lgb
IDS = [1,2,4,5,3]
features = ['Year','Month','Country','Product_ID','Expense_Price','Holiday']
target = 'Sales'
predict = pd.DataFrame(columns=['Year','Month','Country','Product_ID','Expense_Price','Holiday','Sales'])

for i in IDS:
    features = ['Year','Month','Country','Product_ID','Expense_Price','Holiday']
    uni_country = list(train_grouped[train_grouped['Product_ID'] == i]['Country'].value_counts().index)
    print(i)
    print(uni_country)
    if i == 3:
        features.remove('Expense_Price')
    else:
        pass
    if(len(uni_country) == 1):
        features.remove('Country')
    else:
        pass
    for c in uni_country:
        print(c)
        train_temp = train_grouped.loc[train_grouped['Product_ID'] == i].loc[train_grouped['Country'] == c]
        test_temp = test_grouped.loc[test_grouped['Product_ID'] == i].loc[test_grouped['Country'] == c]
        print()
    #     salesScaler = MinMaxScaler().fit(train_temp[['Sales']])
    #     train_temp.loc[:,['Sales']] = salesScaler.transform(train_temp[['Sales']])

    #     expenseScaler = MinMaxScaler().fit(list(train_not3['Expense_Price'].values)+list(test_not3['Expense_Price'].values))
    #     train_not3.loc[:,['Expense_Price']] = expenseScaler.transform(train_not3[['Expense_Price']])
    #     test_temp.loc[:,['Expense_Price']] = expenseScaler.transform(test_temp[['Expense_Price']])
        
        train_X = train_temp[features]
        train_y = train_temp[target].values
        test_X = test_temp[features]
        test_y = test_temp[target].values
        print(test_X.shape)
        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'mape',
            'min_data_in_leaf': 2,
            'learning_rate': .1,
            'num_leaves': 16,
            'max_depth': 0,
        }
        lgb_train = lgb.Dataset(train_X,train_y)
        gbm = lgb.train(params, lgb_train, num_boost_round=20)
        test_y_gbm = gbm.predict(test_X)
        
        regr = linear_model.LinearRegression()
        regr.fit(train_X.values, train_y)
        test_y_lin = regr.predict(test_X.values)
        
        lasso = linear_model.Lasso()
        lasso.fit(train_X.values, train_y)
        test_y_lasso = lasso.predict(test_X.values)
        
        rf = RandomForestRegressor()
        rf.fit(train_X.values, train_y)
        test_y_rf = rf.predict(test_X.values)
        
        xgb = xgboost.XGBRegressor()
        xgb.fit(train_X.values, train_y)
        test_y_xgb = xgb.predict(test_X.values)
        
        test_y = 0.5*test_y_xgb + 0.5*test_y_lasso
        test_temp['Sales'] = list(test_y)
        predict = pd.concat([predict,test_temp],ignore_index=True)

1
[2, 0]
2

(12, 6)
0

(12, 6)
2
[3, 2, 1, 0]
3

(12, 6)
2

(12, 6)
1

(12, 6)
0

(12, 6)
4
[5, 4]
5

(9, 6)
4

(9, 6)
5
[4]
4

(9, 5)
3
[2, 0]
2

(3, 5)
0

(3, 5)


In [141]:
predict.shape

(105, 7)

In [142]:
predict['Country'] = label_en.inverse_transform(list(predict['Country'].values))

/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [143]:
predict['Year'] = predict['Year'].astype('str').astype('int')
predict['Month'] = predict['Month'].astype('str').astype('int')
predict['Product_ID'] = predict['Product_ID'].astype('str').astype('int')

In [144]:
predict = pd.merge(test, predict, how='inner', on=['Year','Month','Country','Product_ID'])
predict = predict[['S_No','Year','Month','Product_ID','Country','Sales_y']]

In [145]:
predict = predict.rename(columns={"Sales_y":"Sales"})
predict.to_csv("subs8.csv", index=False)